In [1]:
%%time
import pal
import json
import random
import math
from pal.prompt import math_prompts, colored_object_prompt, penguin_prompt, date_understanding_prompt, algorithmic_prompt
from tqdm.notebook import tqdm

CPU times: user 296 ms, sys: 20.2 ms, total: 316 ms
Wall time: 454 ms


In [2]:
sum([5, 3, 2])

10

In [4]:
interface = pal.interface.ProgramInterface(
  model='gpt-4o-mini',
  stop='\n\n\n', # stop generation str for Codex API
  get_answer_expr='solution()' # python expression evaluated after generated code to obtain answer 
)

prompt="""Whenever you synthesize information and perform reasoning, it is a Thought. Whenever you need to perform an operation or a function, it is an Action. There are 3 types of Actions:
(1) ADD_START[object;number]ADD_END: ADD takes the object, and a number, and adds the number to a running count of the number of times the object has appeared. For example, ADD_START[trombone;5] would add 5 to the number of times "trombone" has appeared.
(2) SUM_START[]SUM_END: SUM returns the sum of the counts of all objects currently being counted.
(3) FINISH_START[answer]FINISH_END: When you finish the question, write your answer as FINISH_START[answer]FINISH_END and terminate your response.
Terminate your response after an action. Then, an Observation is returned by the user. IMPORTANT: YOU DO NOT WRITE OBSERVATIONS. THEY WILL BE PROVIDED TO YOU. You will be given a question and you will perform Thoughts and Actions until you get the answer. When you get the answer, you will perform the Action called Finish. While you are reasoning out solutions, you will continually refer back to relevant portions of the question and employ logical reasoning to determine your next action. Let's think step by step.
""".strip()
json_list = None
max_count = 2
counter = 0
responses = []
with open('../prompt-lib/data/tasks/objectcounting.jsonl', 'r') as json_file:
    json_list = list(json_file)
jsonSamples = random.sample(range(0, len(json_list)), max_count)
print(jsonSamples)
#jsonSamples = [819, 1000, 699, 749, 580, 366, 1275, 203, 492, 773]
#jsonSamples = [269, 436, 836, 1291, 1283, 712, 391, 474, 1164, 783]


for json_pos in tqdm(jsonSamples, desc="Going through table questions"):
    result = json.loads(json_list[json_pos])
    question = result["input"]
    target = result["target"]
    curResponse = {"question": question, "target": target}
    #print("Question: " + question)
    #print("Target: ", end="")
    #print(target)
    messages=[
        {"role": "system", "content": "You are a helpful assistant that accurately solves reasoning questions about tables by acting as an agent who will perform Thoughts, where you will perform logical reasoning, and Actions, which perform functions or operations."},
        {"role": "user", "content": prompt + '\n' + """Question: There are 15 elephants, 3 pianos, 23 giraffes, and 18 violas in a room. How many animals are there?"""},
        {"role": "assistant", "content": """Thought: Elephants are an animal. Pianos are not an animal. Giraffes are an animal. Violas are not an animal. Thus, the only animals are the elephants and giraffes, so those are the only ones I need to count. I will first add the 15 elephants.
Action: ADD_START[elephant;15]ADD_END"""},
        {"role": "user", "content": "Observation: Added 15 elephant"},
        {"role": "assistant", "content": """Thought: I will now add the 23 giraffes.
Action: ADD_START[giraffe;23]"""},
        {"role": "user", "content": "Observation: Added 23 giraffe"},
        {"role": "assistant", "content": """Thought: Now to find the total number of animals, I need to find the sum of the animals I've put in.
Action: SUM_START[]SUM_END"""},
        {"role": "user", "content": "Observation: 38"},
        {"role": "assistant", "content": """Thought: Thus, there are 38 animals.
Action: FINISH_START[38]FINISH_END"""},
        {"role": "user", "content": prompt + '\n' + question}
    ]
#    for message in messages:
#        print(message["role"].upper())
#        print(message["content"])
#        print("")
#    break
    # interface.verbose = True
    finished = False
    curDict = dict()
    #print(question)
    #print("Target: " + str(target))
    while not finished:
        gens = interface.generate_history(messages=messages, max_tokens=1024)
        output = gens[0]
        #print(output)
        messages.append({"role": "assistant", "content": output})
        curAction = output[output.find("Action: ") + len("Action: "):]
        #print("Action: " + curAction)
        if "finish" in curAction.lower():
            finished = True
            expression = curAction[curAction.find("FINISH_START[") + len("FINISH_START["):curAction.find("]FINISH_END")]
            curResponse["received"] = expression
            #print(expression)
        else:
            evaled = None
            expression = None
            if "ADD" in curAction:
                expression = curAction[curAction.find("ADD_START[") + len("ADD_START["):curAction.find("]ADD_END")]
                expression = expression.split(';')
                evaled = "Added " + expression[1] + " " + expression[0]
                expression[1] = int(expression[1])
                if expression[0] in curDict.keys():
                    curDict[expression[0]] += expression[1]
                else:
                    curDict[expression[0]] = expression[1]
            elif "SUM" in curAction:
                expression = curAction[curAction.find("SUM_START[") + len("SUM_START["):curAction.find("]SUM_END")]
                evaled = str(sum(curDict.values()))
            else:
                evaled = "That was not a valid action. Please input a valid one."
            #print("Expression: ", end = "")
            #print(expression)
            messages.append({"role": "user", "content": "Observation: " + evaled})
            #print("Evaluation: ", end="")
            #print(evaled)
            #print(messages[-1]["content"])
    curResponse["messages"] = messages
    responses.append(curResponse)
    counter += 1
numCorrect = 0

for response in responses:
    #print("Received: " + response["received"])
    #print("Target: ", end="")
    #print(response["target"])
    #print("")
    #for message in response["messages"]:
    #    print(message["content"])
    if response["target"] == response["received"]:
        numCorrect += 1
print("Number correct = " + str(numCorrect))

[554, 580]


Going through table questions:   0%|          | 0/2 [00:00<?, ?it/s]

Number correct = 2


In [10]:
for response in responses:
    #print("Received: " + response["received"])
    #print("Target: ", end="")
    #print(response["target"])
    #print("")
    #for message in response["messages"]:
    #    print(message["content"])
    if response["target"] != response["received"]:
        for message in response["messages"]:
            print(message["content"])

You are a helpful assistant that accurately solves reasoning questions about tables by acting as an agent who will perform Thoughts, where you will perform logical reasoning, and Actions, which perform functions or operations.
Whenever you synthesize information and perform reasoning, it is a Thought. Whenever you need to perform an operation or a function, it is an Action. There are 3 types of Actions:
(1) ADD_START[object;number]ADD_END: ADD takes the object, and a number, and adds the number to a running count of the number of times the object has appeared. For example, ADD_START[trombone;5] would add 5 to the number of times "trombone" has appeared.
(2) SUM_START[]SUM_END: SUM returns the sum of the counts of all objects currently being counted.
(3) FINISH_START[answer]FINISH_END: When you finish the question, write your answer as FINISH_START[answer]FINISH_END and terminate your response.
Terminate your response after an action. Then, an Observation is returned by the user. IMPORT

In [13]:
with open('objectcounting-results-02-gpt-4o-mini.json', 'w') as f:
    json.dump(responses, f)